### import libraries

In [1]:
! pip install netCDF4

     |████████████████████████████████| 4.1MB 4.5MB/s eta 0:00:01
     |████████████████████████████████| 327kB 33.7MB/s eta 0:00:01


In [1]:
import netCDF4 # python API to work with netcdf (.nc) files
import os
import datetime
from osgeo import gdal, ogr, osr
import numpy as np # library to work with matrixes and computations in general
import matplotlib.pyplot as plt # plotting library
from auxiliary_classes import convert_time,convert_time_reverse,kelvin_to_celsius,kelvin_to_celsius_vector,Grid,Image,subImage
import json
import geojson, gdal, subprocess

### auxiliary functions

In [2]:
def print_geojson(tname, tvalue, fname, longitude, latitude, startdoc, position,endloop): #for printing to geojson - start,end,attributes
    fname = fname +".geojson"
    pmode="a"
    if startdoc==1:
        with open(fname, mode="w", encoding='utf-8') as f1: #start of geojson
            tstring = "{\n\"type\": \"FeatureCollection\",\n\"features\": ["
            print(tstring, file=f1)
            f1.close()
    else:
        if position==0: #for printing to geojson - geometry, longitude, latitude
            tstring = "\"type\": \"Feature\",\n\"geometry\": {\n\"type\": \"Point\",\n\"coordinates\": [" + str(longitude) + ","+ str(latitude) + "]\n},\n\"properties\": {"
            fname = fname 
            with open(fname, mode=pmode, encoding='utf-8') as f1:
                print(tstring, file=f1)
                f1.close()
        elif position==1:  #start of point attributes
            with open(fname, mode=pmode, encoding='utf-8') as f1:
                print("{", file=f1)
                f1.close()  
        elif position==2: #print attribute (not last)
             with open(fname, mode=pmode, encoding='utf-8') as f1:
                ttext = "\"" + str(tname) + "\": \"" +str(tvalue) + "\","
                print(ttext, file=f1) 
                f1.close() 
        elif position==3: #print last attribute
            with open(fname, mode=pmode, encoding='utf-8') as f1:
                ttext = "\"" + str(tname) + "\": \"" +str(tvalue) + "\""
                print(ttext, file=f1) 
                f1.close()        
        elif position==4: #end of point attributes
            with open(fname, mode=pmode, encoding='utf-8') as f1:  
                if endloop==0:
                    print("}\n},", file=f1)
                    f1.close()
                else:  #end of geojson
                    print("}\n}\n]\n}", file=f1)
                    f1.close()  

In [3]:
def trend(inputlist, nametrend, namediff, fname): 
    listlong = len(inputlist)
    if listlong <= 1:
        trendcoef = 0
        timediff = 0
        
    else:
        x = np.arange(0,len(inputlist))
        y = inputlist
        z = np.polyfit(x,y,1)
        trendcoef=z[0]
        timediff=int(trendcoef*(listlong-1))
    print_geojson(nametrend, trendcoef, fname, 0, 0, 0, 2, 0)
    print_geojson(namediff, timediff, fname, 0, 0, 0, 3, 0)
    
    
    

In [4]:
def trend2(inputlist, nametrend, namediff, endyear, startyear, fname,fnameavg):
    listlong = endyear-startyear+1
    numberweeks = len(inputlist[0])
    for j in range(0, numberweeks,1):
        tempweek = j +1
        if listlong <= 1:
            trendcoef = 0
            timediff = 0
            
        else:
            x = np.arange(0,listlong)
            y = []
            for i in range(0, listlong, 1): 
                y.append( inputlist[i][j])
            z = np.polyfit(x,y,1)
            trendcoef=z[0]
            timediff=int(trendcoef*(listlong-1))
        nametrend2 = nametrend + str(tempweek)
        namediff2 = namediff + str(tempweek)
        print_geojson(nametrend2, trendcoef, fname, 0, 0, 0, 2, 0)
        print_geojson(nametrend2, trendcoef, fnameavg, 0, 0, 0, 2, 0)
        if j == (numberweeks-1):
            print_geojson(namediff2, timediff, fname, 0, 0, 0, 3, 0)
            print_geojson(namediff2, timediff, fnameavg, 0, 0, 0, 3, 0)
        else:
            print_geojson(namediff2, timediff, fname, 0, 0, 0, 2, 0)
            print_geojson(namediff2, timediff, fnameavg, 0, 0, 0, 2, 0)
            
    
    

In [5]:
def avg2Dlist(inputlist,startyear,endyear): #average for 2D list ->1D list # inputs: inputlist = 2D list, output: avglist = 1D list with avg values
    numberyear = endyear-startyear+1
    listlen = len(inputlist[0])
    templist = []  
    avglist = []  
    for i in range(0, listlen,1):
        for j in range(0, numberyear,1):
            templist.append(inputlist[j][i])
        tempvalue=sum(templist)/len(templist)
        avglist.append(tempvalue) 
        templist = [] 
    return avglist
            

In [6]:
def acumulatelist(inputlist): #average for 2D list ->1D list # inputs: inputlist = 2D list, output: avglist = 1D list with avg values
    listlen = len(inputlist)
    for i in range (0,listlen-1,1):
        inputlist[i+1] += inputlist[i]
    return inputlist

In [7]:
def printlistasweekgeojson(inputlist,name,fname,fnameavg,endloop): # from list of week values print geojson
    listlen = len(inputlist)
    for i in range(0, listlen,1):
        tempvalue=inputlist[i]
        tvarname = name + str(i+1)
        if endloop==1 and i == (listlen-1):
            print_geojson(tvarname, tempvalue, fname, 0, 0, 0, 3, 0)
            print_geojson(tvarname, tempvalue, fnameavg, 0, 0, 0, 2, 0)
        else:
            print_geojson(tvarname, tempvalue, fname, 0, 0, 0, 2, 0)
            print_geojson(tvarname, tempvalue, fnameavg, 0, 0, 0, 2, 0)
        
    

###  Deficits: function for one place

In [8]:
from datetime import date, timedelta
def findprecipitation(allweekevapolist,allmonthevapolist,yearevapolist,allweekrunlist,allmonthrunlist,yearrunlist,allweekdeficlist,allmonthdeficlist,yeardeficlist,latitude,longitude,year,endyear,im,enddate, startdate, fnamepreci, allweekprecilist,precipitationparam, fnameannualpreci, yearprecilist, unitcoeff,fnamepreciaccum,fnamempreciaccum, fnamempreci,allmonthprecilist,fnameevapo,fnameevapoaccum,fnamemevapo,fnamemevapoaccum,fnameavgevaporation,fnameavgmevaporation,fnameannualevapo,fnamerun,fnamerunaccum,fnamemrun,fnamemrunaccum,fnameavgrunoff,fnameavgmrunoff,fnameannualrun,fnamedefic,fnamedeficaccum,fnamemdefic,fnamemdeficaccum,fnameavgdeficits,fnameavgmdeficits,fnameannualdefic):
    
    
    sdate = startdate   # start date for searching last frost date
    edate = enddate   # end date for searching last frost date
    delta = edate - sdate       # as timedelta
    sevendays=0 # for determination of new week (1-7)
    currentweek=1 # for determination of weeks
    starthourday = 0
    endhourday = 23
    sdaylong = str(sdate)
    tmonth = int(sdaylong[5:7])
    currentmonth = tmonth
    weekprecilist = []
    weekprecipitation=0
    weekprecilist = []
    monthprecilist =[]
    
    weekprecisum = 0
    yearprecisum = 0
    monthprecisum = 0
    
    weekevapolist = []
    weekevaporation=0
    weekevapolist = []
    monthevapolist =[]
    
    weekevaposum = 0
    yearevaposum = 0
    monthevaposum = 0
    
    weekrunlist = []
    weekrunoff=0
    weekrunlist = []
    monthrunlist =[]
    
    weekrunsum = 0
    yearrunsum = 0
    monthrunsum = 0
    
        
    weekdeficlist = []
    weekdeficits=0
    weekdeficlist = []
    monthdeficlist =[]
    
    weekdeficsum = 0
    yeardeficsum = 0
    monthdeficsum = 0
    
       
    
    for i in range(delta.days+1):
        daylong = sdate + timedelta(days=i)
        sdaylong = str(daylong)
        tday = int(sdaylong[8:10])
        tmonth = int(sdaylong[5:7])
        tyear = int(sdaylong[0:4])
        
        dayprecisum = 0 # start value
        dayevaposum = 0 # start value
        dayrunsum = 0 # start value
        daydeficsum = 0 # start value
        
        sevendays+=1
        for hour in range(starthourday, endhourday+1, 1): # for specific hours (all day,only sunrise hours,..)
            time=convert_time_reverse(datetime.datetime(tyear, tmonth, tday, hour, 0)) 
            slice_dictionary={'lon':[longitude,],'lat':[latitude],'time':[int(time)]}
            currentpreci=float(im.slice(precipitationparam,slice_dictionary))*unitcoeff 
            currentevapo=float(im.slice(evaporationparam,slice_dictionary))*unitcoeff 
            currentrun=float(im.slice(runoffparam,slice_dictionary))*unitcoeff*0.0001 
            currentdefic=currentpreci+currentevapo
            
            dayprecisum += currentpreci
            yearprecisum += currentpreci
            
            dayevaposum += currentevapo
            yearevaposum += currentevapo
            
            dayrunsum += currentrun
            yearrunsum += currentrun
            
            daydeficsum += currentdefic
            yeardeficsum += currentdefic
            
            
                    
        if daylong == edate: # save month date for last date in season
            monthprecisum+=dayprecisum
            monthprecilist.append(monthprecisum)
            tvarname = "M" + str(year) + "_" + str(tmonth)
            print_geojson(tvarname, monthprecisum, fnamempreci, 0, 0, 0, 2, 0)
            tvarname = "M" + str(year) + "_" + str(tmonth)
            print_geojson(tvarname, yearprecisum, fnamempreciaccum, 0, 0, 0, 2, 0)
            
            monthevaposum+=dayevaposum
            monthevapolist.append(monthevaposum)
            tvarname = "M" + str(year) + "_" + str(tmonth)
            print_geojson(tvarname, monthevaposum, fnamemevapo, 0, 0, 0, 2, 0)
            tvarname = "M" + str(year) + "_" + str(tmonth)
            print_geojson(tvarname, yearevaposum, fnamemevapoaccum, 0, 0, 0, 2, 0)
            
            monthrunsum+=dayrunsum
            monthrunlist.append(monthrunsum)
            tvarname = "M" + str(year) + "_" + str(tmonth)
            print_geojson(tvarname, monthrunsum, fnamemrun, 0, 0, 0, 2, 0)
            tvarname = "M" + str(year) + "_" + str(tmonth)
            print_geojson(tvarname, yearrunsum, fnamemrunaccum, 0, 0, 0, 2, 0)
            
            monthdeficsum+=daydeficsum
            monthdeficlist.append(monthdeficsum)
            tvarname = "M" + str(year) + "_" + str(tmonth)
            print_geojson(tvarname, monthdeficsum, fnamemdefic, 0, 0, 0, 2, 0)
            tvarname = "M" + str(year) + "_" + str(tmonth)
            print_geojson(tvarname, yeardeficsum, fnamemdeficaccum, 0, 0, 0, 2, 0)
        
        
        elif tmonth == currentmonth:
            monthprecisum+=dayprecisum
            
            monthevaposum+=dayevaposum
            
            monthrunsum+=dayrunsum
            
            monthdeficsum+=daydeficsum
        
        else:
            monthprecilist.append(monthprecisum)
            tvarname = "M" + str(year) + "_" + str(tmonth)
            print_geojson(tvarname, monthprecisum, fnamempreci, 0, 0, 0, 2, 0)
            tvarname = "M" + str(year) + "_" + str(tmonth)
            print_geojson(tvarname, yearprecisum, fnamempreciaccum, 0, 0, 0, 2, 0)
            monthprecisum=dayprecisum
            currentmonth=tmonth
            
            monthevapolist.append(monthevaposum)
            tvarname = "M" + str(year) + "_" + str(tmonth)
            print_geojson(tvarname, monthevaposum, fnamemevapo, 0, 0, 0, 2, 0)
            tvarname = "M" + str(year) + "_" + str(tmonth)
            print_geojson(tvarname, yearevaposum, fnamemevapoaccum, 0, 0, 0, 2, 0)
            monthevaposum=dayevaposum
            
            monthrunlist.append(monthrunsum)
            tvarname = "M" + str(year) + "_" + str(tmonth)
            print_geojson(tvarname, monthrunsum, fnamemrun, 0, 0, 0, 2, 0)
            tvarname = "M" + str(year) + "_" + str(tmonth)
            print_geojson(tvarname, yearrunsum, fnamemrunaccum, 0, 0, 0, 2, 0)
            monthrunsum=dayrunsum
            
            monthdeficlist.append(monthdeficsum)
            tvarname = "M" + str(year) + "_" + str(tmonth)
            print_geojson(tvarname, monthdeficsum, fnamemdefic, 0, 0, 0, 2, 0)
            tvarname = "M" + str(year) + "_" + str(tmonth)
            print_geojson(tvarname, yeardeficsum, fnamemdeficaccum, 0, 0, 0, 2, 0)
            monthdeficsum=daydeficsum
            
            
            
                
        if daylong == edate: # save week date for last date in season
            
            weekprecisum+=dayprecisum
            weekprecilist.append(weekprecisum)
            tvarname = "W" + str(year) + "_" + str(currentweek)
            print_geojson(tvarname, weekprecisum, fnamepreci, 0, 0, 0, 2, 0)
            tvarname = "W" + str(year) + "_" + str(currentweek)
            print_geojson(tvarname, yearprecisum, fnamepreciaccum, 0, 0, 0, 2, 0)
            
            weekevaposum+=dayevaposum
            weekevapolist.append(weekevaposum)
            tvarname = "W" + str(year) + "_" + str(currentweek)
            print_geojson(tvarname, weekevaposum, fnameevapo, 0, 0, 0, 2, 0)
            tvarname = "W" + str(year) + "_" + str(currentweek)
            print_geojson(tvarname, yearevaposum, fnameevapoaccum, 0, 0, 0, 2, 0)
            
            weekrunsum+=dayrunsum
            weekrunlist.append(weekrunsum)
            tvarname = "W" + str(year) + "_" + str(currentweek)
            print_geojson(tvarname, weekrunsum, fnamerun, 0, 0, 0, 2, 0)
            tvarname = "W" + str(year) + "_" + str(currentweek)
            print_geojson(tvarname, yearrunsum, fnamerunaccum, 0, 0, 0, 2, 0)
            
            weekdeficsum+=daydeficsum
            weekdeficlist.append(weekdeficsum)
            tvarname = "W" + str(year) + "_" + str(currentweek)
            print_geojson(tvarname, weekdeficsum, fnamedefic, 0, 0, 0, 2, 0)
            tvarname = "W" + str(year) + "_" + str(currentweek)
            print_geojson(tvarname, yeardeficsum, fnamedeficaccum, 0, 0, 0, 2, 0)
            
            
        elif sevendays<=7:  # new week?   
            weekprecisum+=dayprecisum
            
            weekevaposum+=dayevaposum
            
            weekrunsum+=dayrunsum
            
            weekdeficsum+=daydeficsum
            
        else:
            weekprecilist.append(weekprecisum)
            tvarname = "W" + str(year) + "_" + str(currentweek)
            print_geojson(tvarname, weekprecisum, fnamepreci, 0, 0, 0, 2, 0)
            tvarname = "W" + str(year) + "_" + str(currentweek)
            print_geojson(tvarname, yearprecisum, fnamepreciaccum, 0, 0, 0, 2, 0)
            weekprecisum=dayprecisum
            sevendays=0
            currentweek+=1
            
            weekevapolist.append(weekevaposum)
            tvarname = "W" + str(year) + "_" + str(currentweek)
            print_geojson(tvarname, weekevaposum, fnameevapo, 0, 0, 0, 2, 0)
            tvarname = "W" + str(year) + "_" + str(currentweek)
            print_geojson(tvarname, yearevaposum, fnameevapoaccum, 0, 0, 0, 2, 0)
            weekevaposum=dayevaposum
            
            weekrunlist.append(weekrunsum)
            tvarname = "W" + str(year) + "_" + str(currentweek)
            print_geojson(tvarname, weekrunsum, fnamerun, 0, 0, 0, 2, 0)
            tvarname = "W" + str(year) + "_" + str(currentweek)
            print_geojson(tvarname, yearrunsum, fnamerunaccum, 0, 0, 0, 2, 0)
            weekrunsum=dayrunsum
            
            weekdeficlist.append(weekdeficsum)
            tvarname = "W" + str(year) + "_" + str(currentweek)
            print_geojson(tvarname, weekdeficsum, fnamedefic, 0, 0, 0, 2, 0)
            tvarname = "W" + str(year) + "_" + str(currentweek)
            print_geojson(tvarname, yeardeficsum, fnamedeficaccum, 0, 0, 0, 2, 0)
            weekdeficsum=daydeficsum
        
                    
          
    allweekprecilist.append(weekprecilist)
    allmonthprecilist.append(monthprecilist)
    yearprecilist.append(yearprecisum)
    tvarname = "Pr" + str(year) 
    print_geojson(tvarname, yearprecisum, fnameannualpreci, 0, 0, 0, 2, 0)
    
    allweekevapolist.append(weekevapolist)
    allmonthevapolist.append(monthevapolist)
    yearevapolist.append(yearevaposum)
    tvarname = "Ev" + str(year) 
    print_geojson(tvarname, yearevaposum, fnameannualevapo, 0, 0, 0, 2, 0)
    
    allweekrunlist.append(weekrunlist)
    allmonthrunlist.append(monthrunlist)
    yearrunlist.append(yearrunsum)
    tvarname = "Run" + str(year) 
    print_geojson(tvarname, yearrunsum, fnameannualrun, 0, 0, 0, 2, 0)
    
    allweekdeficlist.append(weekdeficlist)
    allmonthdeficlist.append(monthdeficlist)
    yeardeficlist.append(yeardeficsum)
    tvarname = "Pr" + str(year) 
    print_geojson(tvarname, yeardeficsum, fnameannualdefic, 0, 0, 0, 2, 0)
   
   
       
    
    
    
    
                  

### Find deficits: function for selected years

In [9]:
def precipitationyearly(latorder,lonorder,startyear,endyear,endloop,datafolder,fnamepreci,enddatem, startdatem,enddated, startdated,precipitationparam, fnameannualpreci, unitcoeff,fnamepreciaccum,fnameavgprecipitation,fnamempreciaccum, fnamempreci,fnameavgmprecipitation,fnameevapo,fnameevapoaccum,fnamemevapo,fnamemevapoaccum,fnameavgevaporation,fnameavgmevaporation,fnameannualevapo,fnamerun,fnamerunaccum,fnamemrun,fnamemrunaccum,fnameavgrunoff,fnameavgmrunoff,fnameannualrun,fnamedefic,fnamedeficaccum,fnamemdefic,fnamemdeficaccum,fnameavgdeficits,fnameavgmdeficits,fnameannualdefic):
    print_geojson("", "", fnamepreci, 0, 0, 0, 1,0)
    print_geojson("", "", fnamepreciaccum, 0, 0, 0, 1,0)
    print_geojson("", "", fnameannualpreci, 0, 0, 0, 1,0)
    print_geojson("", "", fnameavgprecipitation, 0, 0, 0, 1,0)
    print_geojson("", "", fnamempreci, 0, 0, 0, 1,0)
    print_geojson("", "", fnamempreciaccum, 0, 0, 0, 1,0)
    print_geojson("", "", fnameavgmprecipitation, 0, 0, 0, 1,0)
    
    print_geojson("", "", fnameevapo, 0, 0, 0, 1,0)
    print_geojson("", "", fnameevapoaccum, 0, 0, 0, 1,0)
    print_geojson("", "", fnameannualevapo, 0, 0, 0, 1,0)
    print_geojson("", "", fnameavgevaporation, 0, 0, 0, 1,0)
    print_geojson("", "", fnamemevapo, 0, 0, 0, 1,0)
    print_geojson("", "", fnamemevapoaccum, 0, 0, 0, 1,0)
    print_geojson("", "", fnameavgmevaporation, 0, 0, 0, 1,0)
    
    print_geojson("", "", fnamerun, 0, 0, 0, 1,0)
    print_geojson("", "", fnamerunaccum, 0, 0, 0, 1,0)
    print_geojson("", "", fnameannualrun, 0, 0, 0, 1,0)
    print_geojson("", "", fnameavgrunoff, 0, 0, 0, 1,0)
    print_geojson("", "", fnamemrun, 0, 0, 0, 1,0)
    print_geojson("", "", fnamemrunaccum, 0, 0, 0, 1,0)
    print_geojson("", "", fnameavgmrunoff, 0, 0, 0, 1,0)
    
    print_geojson("", "", fnamedefic, 0, 0, 0, 1,0)
    print_geojson("", "", fnamedeficaccum, 0, 0, 0, 1,0)
    print_geojson("", "", fnameannualdefic, 0, 0, 0, 1,0)
    print_geojson("", "", fnameavgdeficits, 0, 0, 0, 1,0)
    print_geojson("", "", fnamemdefic, 0, 0, 0, 1,0)
    print_geojson("", "", fnamemdeficaccum, 0, 0, 0, 1,0)
    print_geojson("", "", fnameavgmdeficits, 0, 0, 0, 1,0)
    
    endloopyear =0
   
    
    
    
    allweekprecilist=[] # 2D list for all weeks many years
    allmonthprecilist=[] # 2D list for all months many years
    yearprecilist=[] # 2D list for all weeks many years
    
    allweekevapolist=[] # 2D list for all weeks many years
    allmonthevapolist=[] # 2D list for all months many years
    yearevapolist=[] # 2D list for all weeks many years
    
    allweekrunlist=[] # 2D list for all weeks many years
    allmonthrunlist=[] # 2D list for all months many years
    yearrunlist=[] # 2D list for all weeks many years
    
    allweekdeficlist=[] # 2D list for all weeks many years
    allmonthdeficlist=[] # 2D list for all months many years
    yeardeficlist=[] # 2D list for all weeks many years
    
    
    
    for year in range(startyear, endyear+1, 1):
        source = datafolder + '/' + str(year) + '.nc' 
        im=Image(netCDF4.Dataset(source,'r'))   
        longlist = im.get_data().variables['lon'][:]
        latlist= im.get_data().variables['lat'][:]
        longitude = longlist [lonorder]   
        latitude = latlist[latorder]
        if year == startyear:
            print_geojson("", "", fnamepreci, longitude, latitude, 0, 0,0)
            print_geojson("", "", fnameannualpreci, longitude, latitude, 0, 0,0)
            print_geojson("", "", fnamepreciaccum, longitude, latitude, 0, 0,0)
            print_geojson("", "", fnameavgprecipitation, longitude, latitude, 0, 0,0)
            print_geojson("", "", fnamempreci, longitude, latitude, 0, 0,0)
            print_geojson("", "", fnamempreciaccum, longitude, latitude, 0, 0,0)
            print_geojson("", "", fnameavgmprecipitation, longitude, latitude, 0, 0,0)
            
            print_geojson("", "", fnameevapo, longitude, latitude, 0, 0,0)
            print_geojson("", "", fnameannualevapo, longitude, latitude, 0, 0,0)
            print_geojson("", "", fnameevapoaccum, longitude, latitude, 0, 0,0)
            print_geojson("", "", fnameavgevaporation, longitude, latitude, 0, 0,0)
            print_geojson("", "", fnamemevapo, longitude, latitude, 0, 0,0)
            print_geojson("", "", fnamemevapoaccum, longitude, latitude, 0, 0,0)
            print_geojson("", "", fnameavgmevaporation, longitude, latitude, 0, 0,0)
            
            print_geojson("", "", fnamerun, longitude, latitude, 0, 0,0)
            print_geojson("", "", fnameannualrun, longitude, latitude, 0, 0,0)
            print_geojson("", "", fnamerunaccum, longitude, latitude, 0, 0,0)
            print_geojson("", "", fnameavgrunoff, longitude, latitude, 0, 0,0)
            print_geojson("", "", fnamemrun, longitude, latitude, 0, 0,0)
            print_geojson("", "", fnamemrunaccum, longitude, latitude, 0, 0,0)
            print_geojson("", "", fnameavgmrunoff, longitude, latitude, 0, 0,0)
            
            print_geojson("", "", fnamedefic, longitude, latitude, 0, 0,0)
            print_geojson("", "", fnameannualdefic, longitude, latitude, 0, 0,0)
            print_geojson("", "", fnamedeficaccum, longitude, latitude, 0, 0,0)
            print_geojson("", "", fnameavgdeficits, longitude, latitude, 0, 0,0)
            print_geojson("", "", fnamemdefic, longitude, latitude, 0, 0,0)
            print_geojson("", "", fnamemdeficaccum, longitude, latitude, 0, 0,0)
            print_geojson("", "", fnameavgmdeficits, longitude, latitude, 0, 0,0)
            
        if year == endyear:
            endloopyear=1
            
            
        
        enddate=date(year, enddatem, enddated) 
        startdate=date(year, startdatem, startdated) 
                  
        
        
        
                 
            
        findprecipitation(allweekevapolist,allmonthevapolist,yearevapolist,allweekrunlist,allmonthrunlist,yearrunlist,allweekdeficlist,allmonthdeficlist,yeardeficlist,latitude,longitude,year,endyear,im,enddate, startdate, fnamepreci, allweekprecilist,precipitationparam, fnameannualpreci, yearprecilist, unitcoeff,fnamepreciaccum,fnamempreciaccum, fnamempreci,allmonthprecilist,fnameevapo,fnameevapoaccum,fnamemevapo,fnamemevapoaccum,fnameavgevaporation,fnameavgmevaporation,fnameannualevapo,fnamerun,fnamerunaccum,fnamemrun,fnamemrunaccum,fnameavgrunoff,fnameavgmrunoff,fnameannualrun,fnamedefic,fnamedeficaccum,fnamemdefic,fnamemdeficaccum,fnameavgdeficits,fnameavgmdeficits,fnameannualdefic)
    
    
     
    
    
    avgweekprecilist = avg2Dlist(allweekprecilist,startyear,endyear)
    avgmonthprecilist = avg2Dlist(allmonthprecilist,startyear,endyear)
    printlistasweekgeojson(avgweekprecilist,"PrW",fnamepreci,fnameavgprecipitation, 0)
    printlistasweekgeojson(avgmonthprecilist,"PrM",fnamempreci,fnameavgmprecipitation, 0)
    
    avgweekacuprecilist = acumulatelist(avgweekprecilist)
    avgmonthacuprecilist = acumulatelist(avgmonthprecilist)
    
    printlistasweekgeojson(avgweekacuprecilist,"APrW",fnamepreciaccum,fnameavgprecipitation, endloopyear)
    printlistasweekgeojson(avgmonthacuprecilist,"APrM",fnamempreciaccum,fnameavgmprecipitation, endloopyear)
    
    
    avgpreciyear=sum(yearprecilist)/len(yearprecilist)
    print_geojson("AvgPre", avgpreciyear, fnameannualpreci, 0, 0, 0, 2, 0)
    
    nametrend = "AnTrCo"
    namediff = "Andiff"
    trend(yearprecilist, nametrend, namediff,fnameannualpreci)
    
    nametrend = "TrCo"
    namediff = "Diff"
    trend2(allweekprecilist, nametrend, namediff, endyear, startyear, fnamepreci,fnameavgprecipitation)
    trend2(allmonthprecilist, nametrend, namediff, endyear, startyear, fnamempreci,fnameavgmprecipitation)
   
    print_geojson("", "", fnamepreci, 0, 0, 0, 4,endloop)    
    print_geojson("", "", fnameannualpreci, 0, 0, 0, 4,endloop)    
    print_geojson("", "", fnamepreciaccum, 0, 0, 0, 4,endloop)
    print_geojson("", "", fnameavgprecipitation, 0, 0, 0, 4,endloop)
    print_geojson("", "", fnamempreci, 0, 0, 0, 4,endloop)    
    print_geojson("", "", fnamempreciaccum, 0, 0, 0, 4,endloop)
    print_geojson("", "", fnameavgmprecipitation, 0, 0, 0, 4,endloop)
    
    avgweekevapolist = avg2Dlist(allweekevapolist,startyear,endyear)
    avgmonthevapolist = avg2Dlist(allmonthevapolist,startyear,endyear)
    printlistasweekgeojson(avgweekevapolist,"EvW",fnameevapo,fnameavgevaporation, 0)
    printlistasweekgeojson(avgmonthevapolist,"EvM",fnamemevapo,fnameavgmevaporation, 0)
    
    avgweekacuevapolist = acumulatelist(avgweekevapolist)
    avgmonthacuevapolist = acumulatelist(avgmonthevapolist)
    
    printlistasweekgeojson(avgweekacuevapolist,"AEvW",fnameevapoaccum,fnameavgevaporation, endloopyear)
    printlistasweekgeojson(avgmonthacuevapolist,"AEvM",fnamemevapoaccum,fnameavgmevaporation, endloopyear)
    
    
    avgevapoyear=sum(yearevapolist)/len(yearevapolist)
    print_geojson("AvgEv", avgevapoyear, fnameannualevapo, 0, 0, 0, 2, 0)
    
    nametrend = "AnTrCo"
    namediff = "Andiff"
    trend(yearevapolist, nametrend, namediff,fnameannualevapo)
    
    nametrend = "TrCo"
    namediff = "Diff"
    trend2(allweekevapolist, nametrend, namediff, endyear, startyear, fnameevapo,fnameavgevaporation)
    trend2(allmonthevapolist, nametrend, namediff, endyear, startyear, fnamemevapo,fnameavgmevaporation)
   
    print_geojson("", "", fnameevapo, 0, 0, 0, 4,endloop)    
    print_geojson("", "", fnameannualevapo, 0, 0, 0, 4,endloop)    
    print_geojson("", "", fnameevapoaccum, 0, 0, 0, 4,endloop)
    print_geojson("", "", fnameavgevaporation, 0, 0, 0, 4,endloop)
    print_geojson("", "", fnamemevapo, 0, 0, 0, 4,endloop)    
    print_geojson("", "", fnamemevapoaccum, 0, 0, 0, 4,endloop)
    print_geojson("", "", fnameavgmevaporation, 0, 0, 0, 4,endloop)
    
    avgweekrunlist = avg2Dlist(allweekrunlist,startyear,endyear)
    avgmonthrunlist = avg2Dlist(allmonthrunlist,startyear,endyear)
    printlistasweekgeojson(avgweekrunlist,"RW",fnamerun,fnameavgrunoff, 0)
    printlistasweekgeojson(avgmonthrunlist,"RM",fnamemrun,fnameavgmrunoff, 0)
    
    avgweekacurunlist = acumulatelist(avgweekrunlist)
    avgmonthacurunlist = acumulatelist(avgmonthrunlist)
    
    printlistasweekgeojson(avgweekacurunlist,"ARunW",fnamerunaccum,fnameavgrunoff, endloopyear)
    printlistasweekgeojson(avgmonthacurunlist,"ARunM",fnamemrunaccum,fnameavgmrunoff, endloopyear)
    
    
    avgrunyear=sum(yearrunlist)/len(yearrunlist)
    print_geojson("AvgR", avgrunyear, fnameannualrun, 0, 0, 0, 2, 0)
    
    nametrend = "AnTrCo"
    namediff = "Andiff"
    trend(yearrunlist, nametrend, namediff,fnameannualrun)
    
    nametrend = "TrCo"
    namediff = "Diff"
    trend2(allweekrunlist, nametrend, namediff, endyear, startyear, fnamerun,fnameavgrunoff)
    trend2(allmonthrunlist, nametrend, namediff, endyear, startyear, fnamemrun,fnameavgmrunoff)
   
    print_geojson("", "", fnamerun, 0, 0, 0, 4,endloop)    
    print_geojson("", "", fnameannualrun, 0, 0, 0, 4,endloop)    
    print_geojson("", "", fnamerunaccum, 0, 0, 0, 4,endloop)
    print_geojson("", "", fnameavgrunoff, 0, 0, 0, 4,endloop)
    print_geojson("", "", fnamemrun, 0, 0, 0, 4,endloop)    
    print_geojson("", "", fnamemrunaccum, 0, 0, 0, 4,endloop)
    print_geojson("", "", fnameavgmrunoff, 0, 0, 0, 4,endloop)
    
    avgweekdeficlist = avg2Dlist(allweekdeficlist,startyear,endyear)
    avgmonthdeficlist = avg2Dlist(allmonthdeficlist,startyear,endyear)
    printlistasweekgeojson(avgweekdeficlist,"DW",fnamedefic,fnameavgdeficits, 0)
    printlistasweekgeojson(avgmonthdeficlist,"DM",fnamemdefic,fnameavgmdeficits, 0)
    
    avgweekacudeficlist = acumulatelist(avgweekdeficlist)
    avgmonthacudeficlist = acumulatelist(avgmonthdeficlist)
    
    printlistasweekgeojson(avgweekacudeficlist,"ADefW",fnamedeficaccum,fnameavgdeficits, endloopyear)
    printlistasweekgeojson(avgmonthacudeficlist,"ADefM",fnamemdeficaccum,fnameavgmdeficits, endloopyear)
    
    
    avgdeficyear=sum(yeardeficlist)/len(yeardeficlist)
    print_geojson("AvgD", avgdeficyear, fnameannualdefic, 0, 0, 0, 2, 0)
    
    nametrend = "AnTrCo"
    namediff = "Andiff"
    trend(yeardeficlist, nametrend, namediff,fnameannualdefic)
    
    nametrend = "TrCo"
    namediff = "Diff"
    trend2(allweekdeficlist, nametrend, namediff, endyear, startyear, fnamedefic,fnameavgdeficits)
    trend2(allmonthdeficlist, nametrend, namediff, endyear, startyear, fnamemdefic,fnameavgmdeficits)
   
    print_geojson("", "", fnamedefic, 0, 0, 0, 4,endloop)    
    print_geojson("", "", fnameannualdefic, 0, 0, 0, 4,endloop)    
    print_geojson("", "", fnamedeficaccum, 0, 0, 0, 4,endloop)
    print_geojson("", "", fnameavgdeficits, 0, 0, 0, 4,endloop)
    print_geojson("", "", fnamemdefic, 0, 0, 0, 4,endloop)    
    print_geojson("", "", fnamemdeficaccum, 0, 0, 0, 4,endloop)
    print_geojson("", "", fnameavgmdeficits, 0, 0, 0, 4,endloop)
    
    
          

### Find deficits: function for selected latitudes, longitudes

In [10]:
def precipitationplaces(startlat, startlon, endlat, endlon, startyear,endyear,exportfolder,datafolder,fnamepreci1,enddatem, startdatem,enddated, startdated, alllatlonfile, precipitationparam, fnameannualpreci1, unitcoeff,fnamepreciaccum1,fnameavgprecipitation1,fnamempreciaccum1, fnamempreci1,fnameavgmprecipitation1,fnameevapo1,fnameevapoaccum1,fnamemevapo1,fnamemevapoaccum1,fnameavgevaporation1,fnameavgmevaporation1,fnameannualevapo1,fnamerun1,fnamerunaccum1,fnamemrun1,fnamemrunaccum1,fnameavgrunoff1,fnameavgmrunoff1,fnameannualrun1,fnamedefic1,fnamedeficaccum1,fnamemdefic1,fnamemdeficaccum1,fnameavgdeficits1,fnameavgmdeficits1,fnameannualdefic1):
      
        
        fnamepreci= exportfolder + "/" +fnamepreci1
        fnamepreciaccum= exportfolder + "/" +fnamepreciaccum1
        fnameannualpreci= exportfolder + "/" +fnameannualpreci1
        fnameavgprecipitation= exportfolder + "/" +fnameavgprecipitation1
        fnamempreci= exportfolder + "/" +fnamempreci1
        fnamempreciaccum= exportfolder + "/" +fnamempreciaccum1
        fnameavgmprecipitation= exportfolder + "/" +fnameavgmprecipitation1
        
        fnameevapo= exportfolder + "/" +fnameevapo1
        fnameevapoaccum= exportfolder + "/" +fnameevapoaccum1
        fnameannualevapo= exportfolder + "/" +fnameannualevapo1
        fnameavgevaporation= exportfolder + "/" +fnameavgevaporation1
        fnamemevapo= exportfolder + "/" +fnamemevapo1
        fnamemevapoaccum= exportfolder + "/" +fnamemevapoaccum1
        fnameavgmevaporation= exportfolder + "/" +fnameavgmevaporation1
        
        fnamerun= exportfolder + "/" +fnamerun1
        fnamerunaccum= exportfolder + "/" +fnamerunaccum1
        fnameannualrun= exportfolder + "/" +fnameannualrun1
        fnameavgrunoff= exportfolder + "/" +fnameavgrunoff1
        fnamemrun= exportfolder + "/" +fnamemrun1
        fnamemrunaccum= exportfolder + "/" +fnamemrunaccum1
        fnameavgmrunoff= exportfolder + "/" +fnameavgmrunoff1
        
        fnamedefic= exportfolder + "/" +fnamedefic1
        fnamedeficaccum= exportfolder + "/" +fnamedeficaccum1
        fnameannualdefic= exportfolder + "/" +fnameannualdefic1
        fnameavgdeficits= exportfolder + "/" +fnameavgdeficits1
        fnamemdefic= exportfolder + "/" +fnamemdefic1
        fnamemdeficaccum= exportfolder + "/" +fnamemdeficaccum1
        fnameavgmdeficits= exportfolder + "/" +fnameavgmdeficits1
        
        
        
         #start in geojson files:
        print_geojson("", "", fnamepreci, 0, 0, 1, 0,0)
        print_geojson("", "", fnamepreciaccum, 0, 0, 1, 0,0)
        print_geojson("", "", fnameannualpreci, 0, 0, 1, 0,0)
        print_geojson("", "", fnameavgprecipitation, 0, 0, 1, 0,0)
        print_geojson("", "", fnamempreci, 0, 0, 1, 0,0)
        print_geojson("", "", fnamempreciaccum, 0, 0, 1, 0,0)
        print_geojson("", "", fnameavgmprecipitation, 0, 0, 1, 0,0)
        
        print_geojson("", "", fnameevapo, 0, 0, 1, 0,0)
        print_geojson("", "", fnameevapoaccum, 0, 0, 1, 0,0)
        print_geojson("", "", fnameannualevapo, 0, 0, 1, 0,0)
        print_geojson("", "", fnameavgevaporation, 0, 0, 1, 0,0)
        print_geojson("", "", fnamemevapo, 0, 0, 1, 0,0)
        print_geojson("", "", fnamemevapoaccum, 0, 0, 1, 0,0)
        print_geojson("", "", fnameavgmevaporation, 0, 0, 1, 0,0)
        
        print_geojson("", "", fnamerun, 0, 0, 1, 0,0)
        print_geojson("", "", fnamerunaccum, 0, 0, 1, 0,0)
        print_geojson("", "", fnameannualrun, 0, 0, 1, 0,0)
        print_geojson("", "", fnameavgrunoff, 0, 0, 1, 0,0)
        print_geojson("", "", fnamemrun, 0, 0, 1, 0,0)
        print_geojson("", "", fnamemrunaccum, 0, 0, 1, 0,0)
        print_geojson("", "", fnameavgmrunoff, 0, 0, 1, 0,0)
        
        print_geojson("", "", fnamedefic, 0, 0, 1, 0,0)
        print_geojson("", "", fnamedeficaccum, 0, 0, 1, 0,0)
        print_geojson("", "", fnameannualdefic, 0, 0, 1, 0,0)
        print_geojson("", "", fnameavgdeficits, 0, 0, 1, 0,0)
        print_geojson("", "", fnamemdefic, 0, 0, 1, 0,0)
        print_geojson("", "", fnamemdeficaccum, 0, 0, 1, 0,0)
        print_geojson("", "", fnameavgmdeficits, 0, 0, 1, 0,0)
        
        
        
        
        
                
        endloop=0
        
        if alllatlonfile==1:  # if it is calculated for all latitudes and longitudes in input file
            source = datafolder + '/' + str(startyear) + '.nc' 
            im=Image(netCDF4.Dataset(source,'r')) 
            arraylon = im.get_data().variables['lon'][0::]
            arraylat = im.get_data().variables['lat'][0::]
            startlat=0
            startlon=0
            endlon= len(arraylon)-1
            endlat= len(arraylat)-1
        
        for latorder in range(startlat, endlat+1, 1):
            for lonorder in range(startlon, endlon+1, 1):
                if latorder==endlat and lonorder==endlon:
                    endloop=1
                precipitationyearly(latorder,lonorder,startyear,endyear,endloop,datafolder,fnamepreci,enddatem, startdatem,enddated, startdated,precipitationparam, fnameannualpreci, unitcoeff,fnamepreciaccum,fnameavgprecipitation,fnamempreciaccum, fnamempreci,fnameavgmprecipitation,fnameevapo,fnameevapoaccum,fnamemevapo,fnamemevapoaccum,fnameavgevaporation,fnameavgmevaporation,fnameannualevapo,fnamerun,fnamerunaccum,fnamemrun,fnamemrunaccum,fnameavgrunoff,fnameavgmrunoff,fnameannualrun,fnamedefic,fnamedeficaccum,fnamemdefic,fnamemdeficaccum,fnameavgdeficits,fnameavgmdeficits,fnameannualdefic)
       
       
        

## <font color=red>Find difference between precipitation and evapotranspiration: input parameters and launch</font> 

In [ ]:
#Time definition:
startyear=2010 #start year (integer) 
endyear=2019 #end year (integer)  
enddatem = 12 # start date (month) each year
enddated = 31  # start date (day) each year
startdatem = 1 # end date (month) each year
startdated = 1 # end date (day) each year

#Optimalization:
starthourday=0 # integer 0-23
endhourday=23 # integer 0-23

# Difference units:
units = 3  # 1 = m (default), 2 = cm, 3 = mm

#Files/Folders name:
datafolder = "data" #folder with data files (named by year) for each year #string
exportfolder = "export-preci_evapo" #for all files (if each file its folder -> insert name of folder to name of file) #export folder must be created #string

fnamepreci ="weekly_precipitation" #name of created files with week precipitation #string
fnamepreciaccum ="weekly_accum_precipitation" #name of created files with week precipitation #string
fnamempreci ="monthly_precipitation" #name of created files with month precipitation #string
fnamempreciaccum ="monthly_accum_precipitation" #name of created files with month precipitation #string
fnameavgprecipitation ="weekly_avg_precipitation" #name of created files with week precipitation #string
fnameavgmprecipitation ="monthly_avg_precipitation" #name of created files with month precipitation #string
fnameannualpreci ="annualsum_precipitation" #name of created files with annual/seasonal/defined period precipitation #string

fnameevapo ="weekly_evaporation" #name of created files with week evaporation #string
fnameevapoaccum ="weekly_accum_evaporation" #name of created files with week evaporation #string
fnamemevapo ="monthly_evaporation" #name of created files with month evaporation #string
fnamemevapoaccum ="monthly_accum_evaporation" #name of created files with month evaporation #string
fnameavgevaporation ="weekly_avg_evaporation" #name of created files with week evaporation #string
fnameavgmevaporation ="monthly_avg_evaporation" #name of created files with month evaporation #string
fnameannualevapo ="annualsum_evaporation" #name of created files with annual/seasonal/defined period evaporation #string

fnamedefic ="weekly_deficits" #name of created files with week deficits #string
fnamedeficaccum ="weekly_accum_deficits" #name of created files with week deficits #string
fnamemdefic ="monthly_deficits" #name of created files with month deficits #string
fnamemdeficaccum ="monthly_accum_deficits" #name of created files with month deficits #string
fnameavgdeficits ="weekly_avg_deficits" #name of created files with week deficits #string
fnameavgmdeficits ="monthly_avg_deficits" #name of created files with month deficits #string
fnameannualdefic ="annualsum_deficits" #name of created files with annual/seasonal/defined period deficits #string


#Area definition:
alllatlonfile=0 #calculate all latitudes and longitudes in input file (1=yes, 0=no)
# if alllatlonfile!=1 then:
startlat=4 # start number of list of latitudes from used netCDF4 file 
startlon=4 # start number of list of longitudes from used netCDF4 file 
endlat=17 # end number of list of latitudes from used netCDF4 file 
endlon=19 # end number of list of longitudes from used netCDF4 file 

#  data parameter:
precipitationparam = 'tprate'
evaporationparam = 'eow_lwe'
runoffparam = 'ro_NON_CDM' # not used in this algorithm


unitcoeff = 1000
if units == 2:
    unitcoeff = 100000
elif units == 3: 
    unitcoeff = 1000000 


 
# not used in this algorithm:
fnamerun ="weekly_runoff" #name of created files with week runoff #string
fnamerunaccum ="weekly_accum_runoff" #name of created files with week runoff #string
fnamemrun ="monthly_runoff" #name of created files with month runoff #string
fnamemrunaccum ="monthly_accum_runoff" #name of created files with month runoff #string
fnameavgrunoff ="weekly_avg_runoff" #name of created files with week runoff #string
fnameavgmrunoff ="monthly_avg_runoff" #name of created files with month runoff #string
fnameannualrun ="annualsum_runoff" #name of created files with annual/seasonal/defined period runoff #string

precipitationplaces(startlat, startlon, endlat, endlon, startyear,endyear,exportfolder,datafolder,fnamepreci,enddatem, startdatem,enddated, startdated,alllatlonfile,precipitationparam, fnameannualpreci, unitcoeff, fnamepreciaccum,fnameavgprecipitation,fnamempreciaccum, fnamempreci,fnameavgmprecipitation,fnameevapo,fnameevapoaccum,fnamemevapo,fnamemevapoaccum,fnameavgevaporation,fnameavgmevaporation,fnameannualevapo,fnamerun,fnamerunaccum,fnamemrun,fnamemrunaccum,fnameavgrunoff,fnameavgmrunoff,fnameannualrun,fnamedefic,fnamedeficaccum,fnamemdefic,fnamemdeficaccum,fnameavgdeficits,fnameavgmdeficits,fnameannualdefic)









## From geojson to shp

In [26]:
args = ['ogr2ogr', '-f', 'ESRI Shapefile', 'export/shp/weekly_precipitation.shp', 'export/weekly_precipitation.geojson']
subprocess.Popen(args)

In [27]:
args = ['ogr2ogr', '-f', 'ESRI Shapefile', 'export/shp/weekly_accum_precipitation.shp', 'export/weekly_accum_precipitation.geojson']
subprocess.Popen(args)

In [28]:
args = ['ogr2ogr', '-f', 'ESRI Shapefile', 'export/shp/annualsum_precipitation.shp', 'export/annualsum_precipitation.geojson']
subprocess.Popen(args)

In [29]:
args = ['ogr2ogr', '-f', 'ESRI Shapefile', 'export/shp/weekly_avg_precipitation.shp', 'export/weekly_avg_precipitation.geojson']
subprocess.Popen(args)

In [30]:
args = ['ogr2ogr', '-f', 'ESRI Shapefile', 'export/shp/monthly_precipitation.shp', 'export/monthly_precipitation.geojson']
subprocess.Popen(args)

In [31]:
args = ['ogr2ogr', '-f', 'ESRI Shapefile', 'export/shp/monthly_accum_precipitation.shp', 'export/monthly_accum_precipitation.geojson']
subprocess.Popen(args)

In [32]:
args = ['ogr2ogr', '-f', 'ESRI Shapefile', 'export/shp/monthly_avg_precipitation.shp', 'export/monthly_avg_precipitation.geojson']
subprocess.Popen(args)